In [64]:
from pytube import YouTube
from datetime import datetime
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import Playlist
import json
from llama_index.core.node_parser import SentenceSplitter
import os
from urllib.parse import quote_plus
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import StorageContext
from llama_index.core import Settings
from sqlalchemy import create_engine
import json
from llama_index.core.schema import TextNode
from llama_index.llms.ollama import Ollama
from transformers import pipeline
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document 

In [65]:
!pip install yake

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


In [66]:
!pip install llama_index
!pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.


In [67]:
import re
import json
import yake
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube, Playlist


def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_transcript = " ".join([entry['text'] for entry in transcript])
        return full_transcript
    except Exception as e:
        print(f"Error getting transcript for video {video_id}: {str(e)}")
        return ""

def get_video_metadata(video_url):
    try:
        yt = YouTube(video_url)
        
        metadata = {
            'Video ID': yt.video_id,
            'Upload Date': yt.publish_date.date().isoformat() if yt.publish_date else "Unknown",
            'Channel': yt.author,
            'Presenter': "Not specified",
            'Title': yt.title,
            'Length': yt.length
        }
        
        return metadata
    except Exception as e:
        print(f"Error getting metadata for video {video_url}: {str(e)}")
        return None

def clean_transcript(transcript):
    # Remove timestamps, speaker labels, and other non-text elements
    cleaned = re.sub(r'\d+:\d+|\[.*?\]|#\d+', '', transcript)

    # Remove special characters, keeping only alphanumeric, spaces, and basic punctuation
    cleaned = re.sub(r'[^\w\s.,!?-]', '', cleaned)

    # Remove extra whitespace
    cleaned = re.sub(r'\s+', ' ', cleaned)

    # Split into sentences
    sentences = re.split(r'(?<=[.!?])\s+', cleaned)

    # Remove empty sentences and strip whitespace
    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

def extract_keywords(text, num_keywords=5):
    kw_extractor = yake.KeywordExtractor(lan="en", n=1, dedupLim=0.9, top=num_keywords, features=None)
    keywords = kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords]
    
def get_together(full_transcript, metadata, cleaned_sentences):
    metadata['Transcript'] = full_transcript
    metadata['cleaned_text'] = cleaned_sentences
    metadata['topics'] = extract_keywords(full_transcript)
    return metadata

def write_json(metadata_list, output_file):
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(metadata_list, json_file, indent=4, ensure_ascii=False)

def process_playlist(playlist_url, output_file):
    playlist = Playlist(playlist_url)
    all_video_metadata = []

    for video_url in playlist.video_urls:
        print(f"Processing video: {video_url}")
        
        # Step 1: Get video metadata
        metadata = get_video_metadata(video_url)
        if metadata is None:
            print(f"Failed to retrieve metadata for {video_url}")
            continue

        # Step 2: Get transcript
        transcript = get_transcript(metadata['Video ID'])

        # Step 3: Clean transcript
        cleaned_sentences = clean_transcript(transcript)

        # Step 4: Combine metadata, transcript, and extract keywords
        complete_data = get_together(transcript, metadata, cleaned_sentences)

        all_video_metadata.append(complete_data)

    # Step 5: Write all video metadata to JSON file
    write_json(all_video_metadata, output_file)

    return f"Pipeline completed. Processed {len(all_video_metadata)} videos. Output written to {output_file}"


In [68]:

playlist_url = "https://www.youtube.com/watch?v=ehr9PZPDq-0&list=PL5bmbAasRKDI_3Ju4BJpY20UvifFPPFoY"
output_file = "video_metadata.json"
result = process_playlist(playlist_url, output_file)
print(result)

Processing video: https://www.youtube.com/watch?v=ehr9PZPDq-0
Processing video: https://www.youtube.com/watch?v=FXhftCKWj7Y
Pipeline completed. Processed 2 videos. Output written to video_metadata.json


In [69]:
# Load the JSON file containing video data
with open('video_metadata.json', 'r') as file:
    video_data_list = json.load(file)
   
embed_model = HuggingFaceEmbedding(
    model_name="Alibaba-NLP/gte-large-en-v1.5",
    trust_remote_code=True
)
Settings.embed_model = embed_model

llm = Ollama(model='gemma2:2b', request_timeout=900.0, base_url=f"http://10.0.0.180:11434")
Settings.llm = llm

In [70]:

db = {
    'db_host':      '10.0.0.180', 
    'db_port':      '5432', 
    'db_name':      'postgres', 
    'db_user':      'postgres', 
    'db_password':  'P0stGr3sP@ss', 
}

In [71]:

# Set up the vector store
table_name = "embeddings"
vector_store = PGVectorStore.from_params(
    database=db['db_name'],
    host=db['db_host'],
    password=quote_plus(db['db_password']),
    port=db['db_port'],
    user=db['db_user'],
    table_name=table_name,
    embed_dim=1024,  
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

In [72]:
# Create a storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Initialize the sentence splitter for chunking
sentence_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

# Process each video's data
for video_data in video_data_list:
    # Join the cleaned text sentences
    cleaned_text = ' '.join(video_data['cleaned_text'])
    
    # Split the cleaned text into chunks
    chunks = sentence_splitter.split_text(cleaned_text)
    
    # Create TextNodes for each chunk and generate embeddings
    nodes = []
    for i, chunk in enumerate(chunks):
        # Generate embedding for the chunk
        embedding = embed_model.get_text_embedding(chunk)
        
        node = TextNode(
            text=chunk,
            embedding=embedding,
            metadata={
                "Video_ID": video_data['Video ID'],
                "Upload Date": video_data['Upload Date'],
                "Title": video_data['Title'],
                "Topic": video_data.get('topics', []),
                "Presenter": video_data['Presenter'],
                "Channel": video_data['Channel'],
                "Length": video_data['Length'],
                "Chunk_ID": f"{video_data['Video ID']}_chunk_{i}"
            },
        )
        nodes.append(node)
    


In [73]:
nodes

[TextNode(id_='657ac2f0-f1bb-4531-8b6f-c9f5f7da1f97', embedding=[0.004017196130007505, -0.0392024926841259, -0.03471594303846359, -0.025210266932845116, 0.0001330651721218601, -0.006913899444043636, -0.0035901477094739676, -0.016093526035547256, -0.008157897740602493, 0.0664154589176178, -0.009710432030260563, -0.008290309458971024, -0.009122288785874844, -0.0028242715634405613, 0.04195117950439453, -0.02903827838599682, 0.021325455978512764, 0.00425338838249445, -0.017245884984731674, -0.0034970359411090612, -0.01680348441004753, -0.031404733657836914, -0.005731942597776651, 0.030134906992316246, -0.019442660734057426, 0.02758975885808468, -0.012156405486166477, -0.030952682718634605, 0.018457001075148582, 0.009546272456645966, -0.0017547514289617538, -0.013924822211265564, 0.05506003275513649, 0.04005783423781395, -0.012602578848600388, 0.021710669621825218, -0.014351767487823963, -0.04878257215023041, 0.0003074818232562393, 0.0013842260232195258, -0.02644537389278412, -0.04816375300

In [74]:
vector_index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store, 
        storage_context=storage_context,
    )


In [75]:
vector_index.insert_nodes(nodes)

In [76]:
query_engine = vector_index.as_query_engine()

In [77]:
def answer_question(question):
    # Retrieve relevant information
    retrieval_result = query_engine.query(question)
    
    # Generate prompt for Gemma 2:2b
    prompt = f"Based on the following information, please answer the question: '{question}'\n\nInformation: {retrieval_result.response}"
    
    # Get response from Gemma 2:2b
    response = llm.complete(prompt)
    
    return response.text



The video discusses market fluctuations and strategies for investors during potential economic downturns.  It explores the possibility of a recession based on upcoming events like the Federal Reserve's actions and seasonal influences, but highlights potential buying opportunities in dips.  A cautious approach is recommended, focusing on stable sectors with strong fundamentals rather than speculation. The videos emphasize long-term investment strategies and portfolio adjustments to navigate volatility effectively. 



In [78]:
# Example query
question = "What are the 2 main topics discussed in the videos?"
answer = answer_question(question)
print(answer)


The two main topics discussed in the videos are:

1. **Nvidia Stock Performance & Investing Opportunities:** This topic analyzes Nvidia's recent stock performance and whether it presents a good investment opportunity, examining factors such as guidance, market response, AI potential, and comparisons to competitors.
2. **Market Outlook amid Economic Uncertainty:**  This topic focuses on the current economic climate of uncertainty and how this could affect investments, including discussions about elections, Fed policy, recession fears, and advice for navigating a "soft landing." 



In [ ]:
# Example usage
question = "Can you give me a summary of the video content in 3 or 4 sentences?"
answer = answer_question(question)
print(answer)